<a href="https://colab.research.google.com/github/hibadash/-Breast-Ultrasound-Classification-Using-Xception-CNN-BUSI-Dataset/blob/ModelTraining/notebooks/02_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notebook 02 — Entraînement du modèle Xception

Ce notebook contient le code complet pour entraîner le modèle Xception CNN sur le dataset BUSI.

## Objectifs:
- Charger et préparer les données
- Configurer les hyperparamètres (learning rate, batch size, epochs)
- Entraîner le modèle avec callbacks (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau)
- Visualiser les résultats d'entraînement
- Sauvegarder le meilleur modèle

In [ ]:
# Cloner le repo GitHub dans un dossier local "breast_project"
import os

# Vérifie si le dossier "breast_project" existe, sinon clone dedans
if not os.path.exists("breast_project"):
    !git clone https://github.com/hibadash/-Breast-Ultrasound-Classification-Using-Xception-CNN-BUSI-Dataset.git breast_project

# Aller dans le dossier cloné (utiliser le chemin absolu pour éviter les imbrications)
import os
breast_project_path = os.path.abspath("breast_project")
os.chdir(breast_project_path)

# Vérifier qu'on est dans le bon répertoire
print(f"✅ Répertoire courant: {os.getcwd()}")
print(f"✅ Contenu du projet:")
!ls

Cloning into 'breast_project'...
remote: Enumerating objects: 1636, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1636 (delta 24), reused 14 (delta 1), pack-reused 1584 (from 2)
Receiving objects: 100% (1636/1636), 194.53 MiB | 19.70 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Updating files: 100% (1589/1589), done.
/content/breast_project
app  data  notebooks  README.md  requirements.txt  results  src


In [ ]:
# Installer les dépendances si nécessaire
!pip install -q tensorflow>=2.15 numpy pandas matplotlib seaborn scikit-learn

In [ ]:
import sys
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    ReduceLROnPlateau,
    CSVLogger,
    TensorBoard
)

# Afficher les versions
print(f"TensorFlow version: {tf.__version__}")
print(f"Python version: {sys.version}")

# Vérifier la disponibilité du GPU
print(f"\nGPU disponible: {tf.config.list_physical_devices('GPU')}")
if tf.config.list_physical_devices('GPU'):
    print("GPU détecté")
else:
    print("Pas de GPU")


In [ ]:
# =======================
# HYPERPARAMÈTRES D'ENTRAÎNEMENT
# =======================

# Hyperparamètres principaux
LEARNING_RATE = 1e-4  # Taux d'apprentissage initial
BATCH_SIZE = 32  # Taille du batch
EPOCHS = 50  # Nombre maximum d'epochs
IMAGE_SIZE = (224, 224)  # Taille des images

# Callbacks
PATIENCE_EARLY_STOPPING = 10  # Arrêt anticipé si pas d'amélioration
PATIENCE_LR_REDUCTION = 5  # Réduction du LR si pas d'amélioration

# Fine-tuning (optionnel)
FINE_TUNE_AFTER_EPOCHS = 20  # Commencer le fine-tuning après N epochs
FINE_TUNE_LEARNING_RATE = 1e-5  # LR plus faible pour fine-tuning

# Chemins
DATASET_DIR = 'data/Dataset_BUSI'
RESULTS_DIR = 'results'
MODEL_SAVE_PATH = os.path.join(RESULTS_DIR, 'model_xception_best.h5')
MODEL_FINAL_PATH = os.path.join(RESULTS_DIR, 'model_xception_final.h5')

# Créer le dossier results
os.makedirs(RESULTS_DIR, exist_ok=True)

print("Configuration chargée!")
print(f"   - Learning rate: {LEARNING_RATE}")
print(f"   - Batch size: {BATCH_SIZE}")
print(f"   - Epochs max: {EPOCHS}")
print(f"   - Early stopping patience: {PATIENCE_EARLY_STOPPING}")


## Préparation des données

In [ ]:
# Vérifier et corriger preprocess.py AVANT l'import
import os

# Vérifier le contenu actuel de preprocess.py
preprocess_file = 'data/preprocess.py'
if os.path.exists(preprocess_file):
    with open(preprocess_file, 'r') as f:
        content = f.read()
    
    # Vérifier si le fichier a le bon code
    if "DATASET_DIR = os.path.join(_project_root, 'data', 'Dataset_BUSI')" not in content:
        print("⚠️  Correction de preprocess.py...")
        # Remplacer l'ancien code par le nouveau
        import re
        
        # Pattern pour trouver l'ancien DATASET_DIR
        old_patterns = [
            r"DATASET_DIR\s*=\s*['\"]Dataset_BUSI['\"]",
            r"DATASET_DIR\s*=\s*['\"]data/Dataset_BUSI['\"]",
        ]
        
        new_code = """# Chemin du dataset - basé sur l'emplacement de ce fichier
_current_file_dir = os.path.dirname(os.path.abspath(__file__))  
_project_root = os.path.dirname(_current_file_dir) 
DATASET_DIR = os.path.join(_project_root, 'data', 'Dataset_BUSI')

# Debug: Afficher le chemin calculé
print(f"[DEBUG preprocess.py] __file__ = {__file__}")
print(f"[DEBUG preprocess.py] DATASET_DIR = {DATASET_DIR}")
print(f"[DEBUG preprocess.py] DATASET_DIR existe? {os.path.exists(DATASET_DIR)}")"""
        
        # Remplacer
        for pattern in old_patterns:
            content = re.sub(pattern + r'.*?\n', '', content, flags=re.MULTILINE)
        
        # Insérer le nouveau code après BATCH_SIZE
        content = re.sub(
            r'(BATCH_SIZE\s*=\s*\d+)',
            r'\1\n\n' + new_code,
            content
        )
        
        # Sauvegarder
        with open(preprocess_file, 'w') as f:
            f.write(content)
        print("✅ preprocess.py corrigé!")
    else:
        print("✅ preprocess.py est à jour")
else:
    print(f"❌ Fichier {preprocess_file} non trouvé!")

# Vérifier le chemin du dataset
print(f"\n📁 Vérification du dataset...")
print(f"   Répertoire courant: {os.getcwd()}")
print(f"   Chemin dataset (notebook): {DATASET_DIR}")
print(f"   Dataset existe? {os.path.exists(DATASET_DIR)}")

if not os.path.exists(DATASET_DIR):
    print(f"\n❌ ERREUR: Dataset non trouvé dans {DATASET_DIR}")
    raise FileNotFoundError(f"Dataset non trouvé: {DATASET_DIR}")
else:
    print(f"✅ Dataset trouvé!")
    train_path = os.path.join(DATASET_DIR, 'train')
    val_path = os.path.join(DATASET_DIR, 'validation')
    test_path = os.path.join(DATASET_DIR, 'test')
    print(f"   - Train existe: {os.path.exists(train_path)}")
    print(f"   - Validation existe: {os.path.exists(val_path)}")
    print(f"   - Test existe: {os.path.exists(test_path)}")
    print()

# Importer les générateurs de données depuis preprocess.py
from data.preprocess import (
    train_generator,
    val_generator,
    test_generator
)

# Afficher les informations sur le dataset
print("📊 INFORMATIONS SUR LE DATASET")
print("="*60)
print(f"Images d'entraînement: {train_generator.samples}")
print(f"Images de validation: {val_generator.samples}")
print(f"Images de test: {test_generator.samples}")
print(f"\nClasses: {sorted(list(train_generator.class_indices.keys()))}")
print(f"Mapping des classes: {train_generator.class_indices}")
print(f"Taille des images: {IMAGE_SIZE}")
print(f"Batch size: {BATCH_SIZE}")
print("="*60)


## Construction du modèle Xception

In [ ]:
# Importer la fonction de création du modèle
from src.model import load_xception_model

# Créer le modèle
num_classes = len(train_generator.class_indices)
print(f"Création du modèle Xception pour {num_classes} classes...")

model = load_xception_model(
    input_shape=(*IMAGE_SIZE, 3),
    num_classes=num_classes,
    trainable=False  # Commencer avec les couches de base gelées
)

print("Modèle créé avec succès!")
print(f"\n Statistiques du modèle:")
print(f"   - Paramètres totaux: {model.count_params():,}")
print(f"   - Paramètres entraînables: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")

# Afficher un résumé du modèle
model.summary()

## Configuration des callbacks

In [ ]:
# Créer tous les callbacks nécessaires
callbacks = [
    # 1. Sauvegarde du meilleur modèle
    ModelCheckpoint(
        filepath=MODEL_SAVE_PATH,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        save_weights_only=False,
        verbose=1,
        save_fmt='h5'
    ),

    # 2. Early Stopping
    EarlyStopping(
        monitor='val_loss',
        mode='min',
        patience=PATIENCE_EARLY_STOPPING,
        restore_best_weights=True,
        verbose=1,
        min_delta=0.0001
    ),

    # 3. Réduction automatique du learning rate
    ReduceLROnPlateau(
        monitor='val_loss',
        mode='min',
        factor=0.5,  # Réduire le LR de moitié
        patience=PATIENCE_LR_REDUCTION,
        min_lr=1e-7,
        verbose=1
    ),

    # 4. Logger CSV
    CSVLogger(
        filename=os.path.join(RESULTS_DIR, 'training_log.csv'),
        separator=',',
        append=False
    ),

    # 5. TensorBoard (pour Colab, on peut utiliser TensorBoard.dev)
    TensorBoard(
        log_dir=os.path.join(RESULTS_DIR, 'tensorboard_logs'),
        histogram_freq=1,
        write_graph=True,
        update_freq='epoch'
    )
]

print(f" {len(callbacks)} callbacks configurés:")
print("   1. ModelCheckpoint - Sauvegarde du meilleur modèle")
print("   2. EarlyStopping - Arrêt anticipé")
print("   3. ReduceLROnPlateau - Réduction du learning rate")
print("   4. CSVLogger - Log des métriques")
print("   5. TensorBoard - Visualisation avancée")


In [ ]:
# Calculer les steps par epoch
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = val_generator.samples // BATCH_SIZE

print("DÉBUT DE L'ENTRAÎNEMENT")
print("="*60)
print(f"Steps par epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")
print("="*60)
print()

# Lancer l'entraînement
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print("\n ENTRAÎNEMENT TERMINÉ!")


## Sauvegarde du modèle final

In [ ]:
# Sauvegarder le modèle final (après tous les epochs)
model.save(MODEL_FINAL_PATH)
print(f" Modèle final sauvegardé: {MODEL_FINAL_PATH}")

# Sauvegarder l'historique en JSON
history_dict = {}
for key, values in history.history.items():
    history_dict[key] = [float(v) for v in values]

history_path = os.path.join(RESULTS_DIR, 'training_history.json')
with open(history_path, 'w') as f:
    json.dump(history_dict, f, indent=2)

print(f" Historique sauvegardé: {history_path}")


## Visualisation des résultats

In [ ]:
# Créer les graphiques d'entraînement
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Loss
axes[0].plot(history.history['loss'], label='Train Loss', marker='o', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', marker='s', linewidth=2)
axes[0].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Accuracy
axes[1].plot(history.history['accuracy'], label='Train Accuracy', marker='o', linewidth=2)
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s', linewidth=2)
axes[1].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plot_path = os.path.join(RESULTS_DIR, 'training_history_plot.png')
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f" Graphique sauvegardé: {plot_path}")
plt.show()


## Résumé des performances

In [ ]:
# Afficher un résumé des performances
print("\n" + "="*70)
print("RÉSUMÉ DE L'ENTRAÎNEMENT")
print("="*70)

final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
best_val_acc = max(history.history['val_accuracy'])
best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1

print(f"   - Epochs effectués: {len(history.history['loss'])}")
print(f"   - Meilleure validation accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%) - Epoch {best_epoch}")
print(f"   - Accuracy finale (train): {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
print(f"   - Accuracy finale (validation): {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")
print(f"   - Loss finale (train): {history.history['loss'][-1]:.4f}")
print(f"   - Loss finale (validation): {history.history['val_loss'][-1]:.4f}")

# Afficher le learning rate final si disponible
if 'lr' in history.history:
    final_lr = history.history['lr'][-1]
    print(f"   - Learning rate final: {final_lr:.2e}")

print("="*70)
print(f"\n Modèle sauvegardé dans: {MODEL_SAVE_PATH}")
